In [4]:
import pandas as pd
import openai
import time
from tqdm import tqdm

tqdm.pandas()

In [5]:
df = pd.read_csv("data/preprocessed_commercial_area.csv")

In [6]:
OPENAI_API_KEY = ""
openai.api_key = OPENAI_API_KEY

In [7]:
# 일부 행정동에 한해서만 진행
area_lst = {"압구정동", "역삼1동", "역삼2동", "청담동", "수서동", "대치2동"}

df = df[df["행정구역"].apply(lambda area: True if area in area_lst else False)]

prompt_df = df[["시군구", "행정구역"]].copy()

In [25]:
def make_food_store_prompt(data):
    prompt = f"""너는 대한민국 상권 또는 지역에 대해 10년 이상 분석을 했다. 모든 지역에 대한 정보와 특징을 알고 있는 상태이다.

{data["시군구"]} {data["행정구역"]}의 외식업 특징은 다음과 같다.

패스트푸드 전체 점포수는 {data["2022 전체 점포수 외식업 패스트푸드"]}개로 서울 행정동내에서 {"많은편" if data["2022 전체 점포수 외식업 패스트푸드 등급"] <= 20 else ("보통인편" if data["2022 전체 점포수 외식업 패스트푸드 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["전체 점포수 외식업 패스트푸드 변화율"])}개 {"증가" if data["전체 점포수 외식업 패스트푸드 변화율"] > 0 else ("동일" if data["전체 점포수 외식업 패스트푸드 변화율"] == 0 else "감소")}했다.
패스트푸드 프렌차이점 개수는 {data["2022 프렌차이즈 점포수 외식업 패스트푸드"]}개로 서울 행정동내에서 {"많은편" if data["2022 프렌차이즈 점포수 외식업 패스트푸드 등급"] <= 20 else ("보통인편" if data["2022 프렌차이즈 점포수 외식업 패스트푸드 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["프렌차이즈 점포수 외식업 패스트푸드 변화율"])}개 {"증가" if data["프렌차이즈 점포수 외식업 패스트푸드 변화율"] > 0 else ("동일" if data["프렌차이즈 점포수 외식업 패스트푸드 변화율"] == 0 else "감소")}했다.
패스트푸드 일반 점포수는 {data["2022 일반 점포수 외식업 패스트푸드"]}개로 서울 행정동내에서 {"많은편" if data["2022 일반 점포수 외식업 패스트푸드 등급"] <= 20 else ("보통인편" if data["2022 일반 점포수 외식업 패스트푸드 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["일반 점포수 외식업 패스트푸드 변화율"])}개 {"증가" if data["일반 점포수 외식업 패스트푸드 변화율"] > 0 else ("동일" if data["일반 점포수 외식업 패스트푸드 변화율"] == 0 else "감소")}했다.

패스트푸드 신생기업 1년 생존율은 {"높은편" if data["2022 신생기업 1년 생존율 외식업 패스트푸드"] >= 70 else ("보통인편" if data["2022 신생기업 1년 생존율 외식업 패스트푸드"] >= 30 else "낮은편")}이다.
패스트푸드 신생기업 3년 생존율은 {"높은편" if data["2022 신생기업 3년 생존율 외식업 패스트푸드"] >= 70 else ("보통인편" if data["2022 신생기업 3년 생존율 외식업 패스트푸드"] >= 30 else "낮은편")}이다.
패스트푸드 신생기업 5년 생존율은 {"높은편" if data["2022 신생기업 5년 생존율 외식업 패스트푸드"] >= 70 else ("보통인편" if data["2022 신생기업 5년 생존율 외식업 패스트푸드"] >= 30 else "낮은편")}이다.
패스트푸드 최근 10년기준 평균 영업기간은 {"긴 편" if data["2022 최근 10년기준 외식업 패스트푸드 등급"] <= 20 else ("보통인편" if data["2022 최근 10년기준 외식업 패스트푸드 등급"] <= 70 else "짧은편")}이다.

패스트푸드 개업율은 {data["2022 개업률 외식업 패스트푸드"]}%로 서울 행정동내에서 {"높은편" if data["2022 개업률 외식업 패스트푸드 등급"] <= 20 else ("보통인편" if data["2022 개업률 외식업 패스트푸드 등급"] <= 70 else "낮은편")}이다.
패스트푸드 폐업율은 {data["2022 폐업률 외식업 패스트푸드"]}%로 서울 행정동내에서 {"높은편" if data["2022 폐업률 외식업 패스트푸드 등급"] <= 20 else ("보통인편" if data["2022 폐업률 외식업 패스트푸드 등급"] <= 70 else "낮은편")}이다.

커피-음료 전체 점포수는 {data["2022 전체 점포수 외식업 커피-음료"]}개로 서울 행정동내에서 {"많은편" if data["2022 전체 점포수 외식업 커피-음료 등급"] <= 20 else ("보통인편" if data["2022 전체 점포수 외식업 커피-음료 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["전체 점포수 외식업 커피-음료 변화율"])}개 {"증가" if data["전체 점포수 외식업 커피-음료 변화율"] > 0 else ("동일" if data["전체 점포수 외식업 커피-음료 변화율"] == 0 else "감소")}했다.
커피-음료 프렌차이점 개수는 {data["2022 프렌차이즈 점포수 외식업 커피-음료"]}개로 서울 행정동내에서 {"많은편" if data["2022 프렌차이즈 점포수 외식업 커피-음료 등급"] <= 20 else ("보통인편" if data["2022 프렌차이즈 점포수 외식업 커피-음료 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["프렌차이즈 점포수 외식업 커피-음료 변화율"])}개 {"증가" if data["프렌차이즈 점포수 외식업 커피-음료 변화율"] > 0 else ("동일" if data["프렌차이즈 점포수 외식업 커피-음료 변화율"] == 0 else "감소")}했다.
커피-음료 일반 점포수는 {data["2022 일반 점포수 외식업 커피-음료"]}개로 서울 행정동내에서 {"많은편" if data["2022 일반 점포수 외식업 커피-음료 등급"] <= 20 else ("보통인편" if data["2022 일반 점포수 외식업 커피-음료 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["일반 점포수 외식업 커피-음료 변화율"])}개 {"증가" if data["일반 점포수 외식업 커피-음료 변화율"] > 0 else ("동일" if data["일반 점포수 외식업 커피-음료 변화율"] == 0 else "감소")}했다.

커피-음료 신생기업 1년 생존율은 {"높은편" if data["2022 신생기업 1년 생존율 외식업 커피-음료"] >= 70 else ("보통인편" if data["2022 신생기업 1년 생존율 외식업 커피-음료"] >= 30 else "낮은편")}이다.
커피-음료 신생기업 3년 생존율은 {"높은편" if data["2022 신생기업 3년 생존율 외식업 커피-음료"] >= 70 else ("보통인편" if data["2022 신생기업 3년 생존율 외식업 커피-음료"] >= 30 else "낮은편")}이다.
커피-음료 신생기업 5년 생존율은 {"높은편" if data["2022 신생기업 5년 생존율 외식업 커피-음료"] >= 70 else ("보통인편" if data["2022 신생기업 5년 생존율 외식업 커피-음료"] >= 30 else "낮은편")}이다.
커피-음료 최근 10년기준 평균 영업기간은 {"긴 편" if data["2022 최근 10년기준 외식업 커피-음료 등급"] <= 20 else ("보통인편" if data["2022 최근 10년기준 외식업 커피-음료 등급"] <= 70 else "짧은편")}이다.

커피-음료 개업율은 {data["2022 개업률 외식업 커피-음료"]}%로 서울 행정동내에서 {"높은편" if data["2022 개업률 외식업 커피-음료 등급"] <= 20 else ("보통인편" if data["2022 개업률 외식업 커피-음료 등급"] <= 70 else "낮은편")}이다.
커피-음료 폐업율은 {data["2022 폐업률 외식업 커피-음료"]}%로 서울 행정동내에서 {"높은편" if data["2022 폐업률 외식업 커피-음료 등급"] <= 20 else ("보통인편" if data["2022 폐업률 외식업 커피-음료 등급"] <= 70 else "낮은편")}이다.

다음과 같은 특징을 바탕으로 먼저 해당 지역을 대표할만한 별명을 작명해주고 그 아래에 별명 대한 설명을 스토리 형식으로 자세하게 설명해줘.
"""
    return prompt

def make_retail_store_prompt(data):
    prompt = f"""너는 대한민국 상권 또는 지역에 대해 10년 이상 분석을 했다. 모든 지역에 대한 정보와 특징을 알고 있는 상태이다.

{data["시군구"]} {data["행정구역"]}의 소매업 특징은 다음과 같다.

편의점 전체 점포수는 {data["2022 전체 점포수 소매업 편의점"]}개로 서울 행정동내에서 {"많은편" if data["2022 전체 점포수 소매업 편의점 등급"] <= 20 else ("보통인편" if data["2022 전체 점포수 소매업 편의점 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["전체 점포수 소매업 편의점 변화율"])}개 {"증가" if data["전체 점포수 소매업 편의점 변화율"] > 0 else ("동일" if data["전체 점포수 소매업 편의점 변화율"] == 0 else "감소")}했다.
편의점 프렌차이점 개수는 {data["2022 프렌차이즈 점포수 소매업 편의점"]}개로 서울 행정동내에서 {"많은편" if data["2022 프렌차이즈 점포수 소매업 편의점 등급"] <= 20 else ("보통인편" if data["2022 프렌차이즈 점포수 소매업 편의점 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["프렌차이즈 점포수 소매업 편의점 변화율"])}개 {"증가" if data["프렌차이즈 점포수 소매업 편의점 변화율"] > 0 else ("동일" if data["프렌차이즈 점포수 소매업 편의점 변화율"] == 0 else "감소")}했다.
편의점 일반 점포수는 {data["2022 일반 점포수 소매업 편의점"]}개로 서울 행정동내에서 {"많은편" if data["2022 일반 점포수 소매업 편의점 등급"] <= 20 else ("보통인편" if data["2022 일반 점포수 소매업 편의점 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["일반 점포수 소매업 편의점 변화율"])}개 {"증가" if data["일반 점포수 소매업 편의점 변화율"] > 0 else ("동일" if data["일반 점포수 소매업 편의점 변화율"] == 0 else "감소")}했다.

편의점 신생기업 1년 생존율은 {"높은편" if data["2022 신생기업 1년 생존율 소매업 편의점"] >= 70 else ("보통인편" if data["2022 신생기업 1년 생존율 소매업 편의점"] >= 30 else "낮은편")}이다.
편의점 신생기업 3년 생존율은 {"높은편" if data["2022 신생기업 3년 생존율 소매업 편의점"] >= 70 else ("보통인편" if data["2022 신생기업 3년 생존율 소매업 편의점"] >= 30 else "낮은편")}이다.
편의점 신생기업 5년 생존율은 {"높은편" if data["2022 신생기업 5년 생존율 소매업 편의점"] >= 70 else ("보통인편" if data["2022 신생기업 5년 생존율 소매업 편의점"] >= 30 else "낮은편")}이다.
편의점 최근 10년기준 평균 영업기간은 {"긴 편" if data["2022 최근 10년기준 소매업 편의점 등급"] <= 20 else ("보통인편" if data["2022 최근 10년기준 소매업 편의점 등급"] <= 70 else "짧은편")}이다.

편의점 개업율은 {data["2022 개업률 소매업 편의점"]}%로 서울 행정동내에서 {"높은편" if data["2022 개업률 소매업 편의점 등급"] <= 20 else ("보통인편" if data["2022 개업률 소매업 편의점 등급"] <= 70 else "낮은편")}이다.
편의점 폐업율은 {data["2022 폐업률 소매업 편의점"]}%로 서울 행정동내에서 {"높은편" if data["2022 폐업률 소매업 편의점 등급"] <= 20 else ("보통인편" if data["2022 폐업률 소매업 편의점 등급"] <= 70 else "낮은편")}이다.

주유소 전체 점포수는 {data["2022 전체 점포수 소매업 주유소"]}개로 서울 행정동내에서 {"많은편" if data["2022 전체 점포수 소매업 주유소 등급"] <= 20 else ("보통인편" if data["2022 전체 점포수 소매업 주유소 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["전체 점포수 소매업 주유소 변화율"])}개 {"증가" if data["전체 점포수 소매업 주유소 변화율"] > 0 else ("동일" if data["전체 점포수 소매업 주유소 변화율"] == 0 else "감소")}했다.
주유소 프렌차이점 개수는 {data["2022 프렌차이즈 점포수 소매업 주유소"]}개로 서울 행정동내에서 {"많은편" if data["2022 프렌차이즈 점포수 소매업 주유소 등급"] <= 20 else ("보통인편" if data["2022 프렌차이즈 점포수 소매업 주유소 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["프렌차이즈 점포수 소매업 주유소 변화율"])}개 {"증가" if data["프렌차이즈 점포수 소매업 주유소 변화율"] > 0 else ("동일" if data["프렌차이즈 점포수 소매업 주유소 변화율"] == 0 else "감소")}했다.
주유소 일반 점포수는 {data["2022 일반 점포수 소매업 주유소"]}개로 서울 행정동내에서 {"많은편" if data["2022 일반 점포수 소매업 주유소 등급"] <= 20 else ("보통인편" if data["2022 일반 점포수 소매업 주유소 등급"] <= 70 else "적은편")}이며, 작년 대비 {abs(data["일반 점포수 소매업 주유소 변화율"])}개 {"증가" if data["일반 점포수 소매업 주유소 변화율"] > 0 else ("동일" if data["일반 점포수 소매업 주유소 변화율"] == 0 else "감소")}했다.

주유소 신생기업 1년 생존율은 {"높은편" if data["2022 신생기업 1년 생존율 소매업 주유소"] >= 70 else ("보통인편" if data["2022 신생기업 1년 생존율 소매업 주유소"] >= 30 else "낮은편")}이다.
주유소 신생기업 3년 생존율은 {"높은편" if data["2022 신생기업 3년 생존율 소매업 주유소"] >= 70 else ("보통인편" if data["2022 신생기업 3년 생존율 소매업 주유소"] >= 30 else "낮은편")}이다.
주유소 신생기업 5년 생존율은 {"높은편" if data["2022 신생기업 5년 생존율 소매업 주유소"] >= 70 else ("보통인편" if data["2022 신생기업 5년 생존율 소매업 주유소"] >= 30 else "낮은편")}이다.
주유소 최근 10년기준 평균 영업기간은 {"긴 편" if data["2022 최근 10년기준 소매업 주유소 등급"] <= 20 else ("보통인편" if data["2022 최근 10년기준 소매업 주유소 등급"] <= 70 else "짧은편")}이다.

주유소 개업율은 {data["2022 개업률 소매업 주유소"]}%로 서울 행정동내에서 {"높은편" if data["2022 개업률 소매업 주유소 등급"] <= 20 else ("보통인편" if data["2022 개업률 소매업 주유소 등급"] <= 70 else "낮은편")}이다.
주유소 폐업율은 {data["2022 폐업률 소매업 주유소"]}%로 서울 행정동내에서 {"높은편" if data["2022 폐업률 소매업 주유소 등급"] <= 20 else ("보통인편" if data["2022 폐업률 소매업 주유소 등급"] <= 70 else "낮은편")}이다.

다음과 같은 특징을 바탕으로 먼저 해당 지역을 대표할만한 별명을 작명해주고 그 아래에 별명 대한 설명을 스토리 형식으로 자세하게 설명해줘."""
    return prompt

def make_population_prompt(data):
    prompt = f"""너는 대한민국 상권 또는 지역에 대해 10년 이상 분석을 했다. 모든 지역에 대한 정보와 특징을 알고 있는 상태이다.

{data["시군구"]} {data["행정구역"]}의 특징은 다음과 같다.

길단위 유동인구 수는 약 {round(data["2022 길단위 유동인구"], 3)}명이며, 서울 행정동내에서 {"많은편" if data["2022 길단위 유동인구 등급"] <= 20 else ("보통인편" if data["2022 길단위 유동인구 등급"] <= 70 else "적은편")}이다.
주거인구 수는 약 {round(data["2022 주거인구"], 3)}명이며, 서울 행정동내에서 {"많은편" if data["2022 주거인구 등급"] <= 20 else ("보통인편" if data["2022 주거인구 등급"] <= 70 else "적은편")}이다.
직장인구 수는 약 {round(data["2022 직장인구"], 3)}명이며, 서울 행정동내에서 {"많은편" if data["2022 직장인구 등급"] <= 20 else ("보통인편" if data["2022 직장인구 등급"] <= 70 else "적은편")}이다.

소득분위는 {data["2022 소득분위"].split(":")[0]}로 {data["2022 소득분위"].split(":")[1]}사이이다.
가구수는 {data["2022 가구수"]}이며 서울 행정동내에서 {"많은편" if data["2022 가구수 등급"] <= 20 else ("보통인편" if data["2022 가구수 등급"] <= 70 else "적은편")}이다.

1평당 월 임대시세는 서울 행정동내에서 {"높은편" if data["2022 환산임대료 전체 등급"] <= 20 else ("보통인편" if data["2022 환산임대료 전체 등급"] <= 70 else "낮은편")}이다.
직년 대비 1평당 월 임대시세는 {abs(data["환산임대료 전체 변화율"])}% {"증가" if data["환산임대료 전체 변화율"] >= 0 else "감소"}했다.

다음과 같은 특징을 바탕으로 먼저 해당 지역을 대표할만한 별명을 작명해주고 그 아래에 별명 대한 설명을 스토리 형식으로 자세하게 설명해줘."""
    
    return prompt

def make_total_story_prompt(data):
    prompt = f"""너는 대한민국 상권 또는 지역에 대해 10년 이상 분석을 했다. 모든 지역에 대한 정보와 특징을 알고 있는 상태이다.

{data["시군구"]} {data["행정구역"]}의 특징은 다음과 같다.

{data["population_story"]}

{data["food_store_story"]}

{data["retail_store_story"]}

해당 지역의 특징이 다음과 같을때 해당 지역을 대표할만한 별명을 작명해주고 그 아래에 별명 대한 설명을 스토리 형식으로 자세하게 설명해줘."""
    
    return prompt

In [9]:
prompt_df["food_store_prompt"] = df.apply(make_food_store_prompt, axis=1)
prompt_df["retail_store_prompt"] = df.apply(make_retail_store_prompt, axis=1)
prompt_df["population_prompt"] = df.apply(make_population_prompt, axis=1)

In [22]:
def get_hashtag(prompt):
    model = "gpt-3.5-turbo"
    messages = [
        {"role": "user", "content": prompt},
    ]
    time.sleep(30)
    response = openai.ChatCompletion.create(model=model, messages=messages)
    return response["choices"][0]["message"]["content"]

In [106]:
prompt_df["food_store_story"] = prompt_df["food_store_prompt"].progress_apply(get_hashtag)
prompt_df["retail_store_story"] = prompt_df["retail_store_prompt"].progress_apply(get_hashtag)
prompt_df["population_story"] = prompt_df["population_prompt"].progress_apply(get_hashtag)

100%|██████████| 6/6 [04:25<00:00, 44.29s/it]


In [18]:
prompt_df.columns

Index(['시군구', '행정구역', 'food_store_prompt', 'retail_store_prompt',
       'population_prompt', 'food_store_story', 'retail_store_story',
       'population_story'],
      dtype='object')

In [20]:
prompt_df["total_story_prompt"] = prompt_df.apply(make_total_story_prompt, axis=1)

In [26]:
prompt_df["total_story"] = prompt_df["total_story_prompt"].progress_apply(get_hashtag)

100%|██████████| 6/6 [04:03<00:00, 40.56s/it]


In [27]:
prompt_df.to_csv("data/prompt_story.csv",index=False)